# Tool Window Usage Analysis – Notebook (English)\nInteractive version of the main script.

In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns\nfrom scipy import stats\nfrom lifelines import KaplanMeierFitter\nfrom lifelines.statistics import logrank_test\nsns.set_style('whitegrid'); plt.rcParams['figure.figsize']=(11,5)\nprint('✅ Libraries ready')

In [ ]:
import os, json\nDATA = '../data/toolwindow_data.csv'  # change if needed\ndf = pd.read_csv(DATA)\ndf.columns = [c.lower().strip() for c in df.columns]\nif 'event' in df.columns and 'event_id' not in df.columns:\n    df = df.rename(columns={'event':'event_id'})\nfor col in ['event_id','open_type','user_id']:\n    if col in df.columns:\n        df[col] = df[col].astype(str).str.strip().str.lower()\ndf['event_id'] = df['event_id'].replace({'opened':'open','closed':'close'})\ndf['timestamp'] = df['timestamp'].astype('int64')\ndf.head()

In [ ]:
def match(df, policy='close_previous'):\n    episodes=[]\n    df=df.sort_values(['user_id','timestamp']).reset_index(drop=True)\n    end_by_user=df.groupby('user_id')['timestamp'].max()\n    for uid, g in df.groupby('user_id'):\n        g=g.reset_index(drop=True)\n        pending=None\n        for _,r in g.iterrows():\n            if r['event_id']=='open':\n                if pending is not None:\n                    if policy=='close_previous':\n                        dur=r['timestamp']-pending['open_ts']\n                        if dur>=0:\n                            episodes.append(dict(user_id=uid,open_timestamp=pending['open_ts'],close_timestamp=r['timestamp'],duration_ms=dur,open_type=pending['open_type'],censored=False))\n                        pending=dict(open_ts=r['timestamp'],open_type=r.get('open_type'))\n                    elif policy=='drop_previous':\n                        pending=dict(open_ts=r['timestamp'],open_type=r.get('open_type'))\n                    else: # keep_first\n                        pass\n                else:\n                    pending=dict(open_ts=r['timestamp'],open_type=r.get('open_type'))\n            elif r['event_id']=='close':\n                if pending is not None:\n                    dur=r['timestamp']-pending['open_ts']\n                    if dur>=0:\n                        episodes.append(dict(user_id=uid,open_timestamp=pending['open_ts'],close_timestamp=r['timestamp'],duration_ms=dur,open_type=pending['open_type'],censored=False))\n                    pending=None\n        if pending is not None:\n            end=end_by_user.loc[uid]\n            dur=end-pending['open_ts']\n            if dur>=0:\n                episodes.append(dict(user_id=uid,open_timestamp=pending['open_ts'],close_timestamp=None,duration_ms=dur,open_type=pending['open_type'],censored=True))\n    epi=pd.DataFrame(episodes)\n    if not epi.empty:\n        epi['duration_seconds']=epi['duration_ms']/1000\n    return epi\n\nepisodes = match(df, 'close_previous')\nepisodes = episodes[episodes['open_type'].notna()]\nepisodes.head()

In [ ]:
def iqr_clean(g, min_ms=100, k=3.0):\n    g=g[g['duration_ms']>=min_ms]\n    Q1=g['duration_seconds'].quantile(0.25); Q3=g['duration_seconds'].quantile(0.75)\n    IQR=Q3-Q1; up=Q3+k*IQR\n    return g[g['duration_seconds']<=up]\n\nclean = episodes.groupby('open_type', group_keys=False).apply(iqr_clean)\nclean.shape

In [ ]:
manual=clean.loc[clean['open_type']=='manual','duration_seconds']\nauto=clean.loc[clean['open_type']=='auto','duration_seconds']\nU,p=stats.mannwhitneyu(manual,auto,alternative='two-sided')\nU,p

In [1]:
plt.figure();\nfor lab in ['manual','auto']:\n    s=np.sort(clean.loc[clean['open_type']==lab,'duration_seconds'])\n    if s.size:\n        y=np.arange(1,s.size+1)/s.size\n        plt.plot(s,y,label=lab)\nplt.legend(); plt.xlabel('seconds'); plt.ylabel('ECDF'); plt.title('ECDF'); plt.show()\n\nkm=KaplanMeierFitter(); plt.figure()\nfor lab in ['manual','auto']:\n    g=clean[clean['open_type']==lab]\n    if not g.empty:\n        km.fit(g['duration_seconds'], event_observed=(~g['censored']).astype(int), label=lab)\n        km.plot()\nplt.title('KM curves')\nplt.xlabel('seconds'); plt.ylabel('survival'); plt.show()

SyntaxError: unexpected character after line continuation character (538422881.py, line 1)